# Build an RNN

Use the techniques discussed in class to build an RNN model.

Remember, this example comes from the textbook *Deep Learning with Python (Second Edition)*. I will be using examples from this textbook throughout the second half of the course. Although not required, you may wish to consult this textbook. The text is provided for free online through Seneca Libraries [here](https://learning-oreilly-com.libaccess.senecacollege.ca/library/view/deep-learning-with/9781617296864/Text/title.xhtml). All of the code from the textbook is on Github [here](https://github.com/fchollet/deep-learning-with-python-notebooks). Any students who wish to work with deep learning models professionally should definitely consider reading the book from cover-to-cover.

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  67.0M      0  0:00:01  0:00:01 --:--:-- 67.0M


In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Found 25000 files belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 168s 262ms/step - loss: 0.5622 - accuracy: 0.7085 - val_loss: 0.4598 - val_accuracy: 0.8340
Epoch 2/10
625/625 [==============================] - 162s 259ms/step - loss: 0.3855 - accuracy: 0.8601 - val_loss: 0.3271 - val_accuracy: 0.8690
Epoch 3/10
625/625 [==============================] - 163s 260ms/step - loss: 0.3041 - accuracy: 0.8953 - val_loss: 0.3637 - val_accuracy: 0.8434
Epoch 4/10
625/625 [==============================] - 163s 260ms/step - loss: 0.2649 - accuracy: 0.9089 - val_loss: 0.3442 - val_accuracy: 0.8482
Epoch 5/10
625/625 [==============================] - 161s 258ms/step - loss: 0.2318 - accuracy: 0.9198 - val_loss: 0.5193 - val_accuracy: 0.8724
Epoch 6/10
625/625 [==============================] - 161s 258ms/step - loss: 0.2057 - accuracy: 0.9329 - val_loss: 0.2842 - val_accuracy: 0.8870
Epoch 7/10
625/625 [==============================] - 161s 258ms/step - loss: 0.1834 - accuracy: 0.9398 - val_loss: 0.3304 -

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________



---



---

# **A secondary RNN model was developed utilizing a comparable methodology as recommended in the textbook.**


In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing import sequence

In [ ]:
# Load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
# Pad sequences
max_review_length = 600
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [ ]:
# Define the RNN model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(20000, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 600, 32)           640000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 693,301
Trainable params: 693,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
391/391 [==============================] - 82s 191ms/step - loss: 0.4518 - accuracy: 0.7799 - val_loss: 0.3289 - val_accuracy: 0.8634
Epoch 2/10
391/391 [==============================] - 42s 108ms/step - loss: 0.2343 - accuracy: 0.9106 - val_loss: 0.3176 - val_accuracy: 0.8709
Epoch 3/10
391/391 [==============================] - 34s 86ms/step - loss: 0.1643 - accuracy: 0.9409 - val_loss: 0.3469 - val_accuracy: 0.8542
Epoch 4/10
391/391 [==============================] - 25s 64ms/step - loss: 0.1216 - accuracy: 0.9579 - val_loss: 0.3884 - val_accuracy: 0.8558
Epoch 5/10
391/391 [==============================] - 25s 63ms/step - loss: 0.1563 - accuracy: 0.9396 - val_loss: 0.4282 - val_accuracy: 0.8560
Epoch 6/10
391/391 [==============================] - 22s 56ms/step - loss: 0.0888 - accuracy: 0.9688 - val_loss: 0.4447 - val_accuracy: 0.8533
Epoch 7/10
391/391 [==============================] - 19s 48ms/step - loss: 0.0554 - accuracy: 0.9825 - val_loss: 0.5345 - val_accurac

In [ ]:
# Convert a new review into a format the model can understand
new_review = "This movie is really good I enjoyed it a lot"
new_review = [imdb.get_word_index()[word] for word in new_review.lower().split()]

new_review = sequence.pad_sequences([new_review], maxlen=max_review_length)

# Use the model to predict the sentiment of the new review
prediction = model.predict(new_review)

# Print the prediction
if prediction > 0.5:
    print("Positive review")
else:
    print("Negative review")

1/1 [==============================] - 0s 381ms/step
Positive review
